# Finding the nearest train/tube station

The purpose of this project is to create module that takes a London postcode input e.g. N4 4AF and will output the nearest train station to it.

In [2]:
import pandas as pd
import requests
from geopy import distance

In [3]:
# A csv file of london stations can be downloaded from this link
# https://www.doogal.co.uk/london_stations.php

# TODO join station data to tube line
london_stations_df = pd.read_csv('london_stations.csv')

In [3]:
LONDON_STATIONS = []
for x in london_stations_df[['Station', 'Latitude', 'Longitude']].values:
    LONDON_STATIONS.append((x[0],(x[1],x[2])))

In [4]:
london_stations_df.head(2)

,Station,Postcode,Latitude,Longitude,TLC,NLC,Owner,Entries and exits 2020,Entries and exits 2019,Entries and exits 2018,...,Interchanges 2014,Interchanges 2013,Interchanges 2012,Interchanges 2011,Interchanges 2010,Interchanges 2009,Interchanges 2008,Interchanges 2007,Interchanges 2006,Interchanges 2005
0,Abbey Wood,SE2 9RH,51.491061,0.121394,ABW,5131,TfL Rail,3825206,3769402.0,3124856.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aber,CF83 1AQ,51.574969,-3.229829,ABE,3813,TfW Rail,228480,245218.0,251108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# test_postcodes = ['N15 3AD', 'N4 4AF', 'N10 3QS', 'N15 4AR', 'N15 4JF']

In [6]:
# TODO refactor - use less code
# TODO Add documentation
# TODO experiment using getters and setters for km?
class NearestStation:
        
    def __init__(self, postcode, unit='m'):
        self.postcode = postcode.lower()
        self.lat_lon = self.geo_dict['lat_lon']
        self._unit = unit
        
    @property
    def unit(self):
        return self._unit
    
    @unit.setter
    def unit(self, new_unit):
        if new_unit in ['m', 'km']:
            self._unit = new_unit
        else:
            raise ValueError('Enter "m" for miles or "km" for kilometers')
        
    @property
    def endpoint(self):
        pc = self.postcode.replace(' ','')
        return f'http://api.postcodes.io/postcodes/{pc}'
    
    @property
    def geo_dict(self):
        geocoded_dict = {}
        r = requests.get(self.endpoint)
        geocoded_dict['postcode'] = r.json()['result']['postcode']
        geocoded_dict['lat_lon'] = (r.json()['result']['latitude'], r.json()['result']['longitude'])
        return geocoded_dict
    
    def distance_miles(self, x):
        return round(distance.distance(self.lat_lon, x).miles,1)
    
    def distance_km(self, x):
        return round(distance.distance(self.lat_lon, x).km,1)
    
    def find_nearest_station(self): 
        #TODO give user option to enter their own list of tuples with locations - new function?
        #TODO can you refactor these further so it's even less code
        if self._unit == 'm':
            distances = [(x[0], self.distance_miles(x[1])) for x in LONDON_STATIONS]
            distances.sort(key=lambda x:x[1])
            return (distances[0])
        elif self._unit == 'km':
            distances = [(x[0], self.distance_km(x[1])) for x in LONDON_STATIONS]
            distances.sort(key=lambda x:x[1])
            return (distances[0])
    
    def find_nearest_n_stations(self,n):
        if self._unit == 'm':
            distances = [(x[0], self.distance_miles(x[1])) for x in LONDON_STATIONS]
            distances.sort(key=lambda x:x[1])
            return (distances[:n])
        elif self._unit == 'km':
            distances = [(x[0], self.distance_km(x[1])) for x in LONDON_STATIONS]
            distances.sort(key=lambda x:x[1])
            return (distances[:n])

In [7]:
x = NearestStation('N4 4AF')

In [8]:
x.geo_dict

{'postcode': 'N4 4AF', 'lat_lon': (51.573188, -0.10576)}

In [9]:
x.lat_lon

(51.573188, -0.10576)

In [10]:
x.endpoint

'http://api.postcodes.io/postcodes/n44af'

In [11]:
x.postcode

'n4 4af'

In [12]:
print(x.find_nearest_station(), "\n", x.unit)

('Harringay', 0.3) 
 m


In [13]:
x.unit

'm'

In [14]:
x.unit ='km' #should change unit

In [15]:
x.unit

'km'

In [16]:
print(x.find_nearest_station(), "\n", x.unit)

('Harringay', 0.5) 
 km


In [17]:
x.find_nearest_n_stations(10)

[('Harringay', 0.5),
 ('Harringay Green Lanes', 0.7),
 ('Crouch Hill', 0.8),
 ('Finsbury Park', 1.0),
 ('Hornsey', 1.5),
 ('Stamford Hill', 2.0),
 ('Upper Holloway', 2.0),
 ('Drayton Park', 2.3),
 ('Seven Sisters', 2.3),
 ('Stoke Newington', 2.4)]

In [18]:
NearestStation('e17 3ss').find_nearest_station()

('Wood Street', 0.4)

In [19]:
# Clean station data?
NearestStation('e17 3ss').find_nearest_n_stations(5)

[('Wood Street', 0.4),
 ('Highams Park', 1.1),
 ('Walthamstow Central', 1.1),
 ("Walthamstow Queen's Road", 1.3),
 ('Leyton Midland Road', 1.6)]